In [2]:

import numpy as np 
import pandas as pd 

import os
import time
import cv2
import glob
import torch 
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
print(os.listdir("./input"))


['cat_to_name.json', 'test set', 'flower_data']


In [53]:
labels= pd.read_json("./input/cat_to_name.json", typ='series')
print("No of Flower Labels : "+str(len(labels)))
all_labels=[]
for i in range(len(labels)):
    all_labels.append(labels[i+1])
print(all_labels)

No of Flower Labels : 102
['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 

In [77]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

data_dir='./input/flower_data/flower_data'

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=test_transforms)
test_data = datasets.ImageFolder('./input/test set/', transform=test_transforms)

testloader = torch.utils.data.DataLoader(test_data)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

In [55]:
def get_unique_data():
    unique_images=[]
    unique_labels=[]
    for image,label in train_data:
        if label not in unique_labels:
            unique_images.append(image)
            unique_labels.append(label)
    return unique_images,unique_labels

In [56]:
import torchvision
unique_images,unique_labels=get_unique_data()
grid=torchvision.utils.make_grid(unique_images,nrow=20, padding=2)
plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid,(1,2,0)))
print('labels:',unique_labels)

KeyboardInterrupt: 

In [ ]:
image_path=data_dir+"/train/1/image_06745.jpg"
image=cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
print(image.shape)

In [21]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [11]:
from torch import nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.Linear(256, 102))

criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.to(device);

cpu


In [23]:
def train_model(model, criterion, optimizer, num_epochs=10):
    since = time.time()
    i=0
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train(True)
                dataloader=trainloader
            else:
                model.train(False)
                dataloader=validloader

            running_loss = 0.0
            running_corrects = 0

            for data in dataloader:
                inputs, labels = data

                inputs=inputs.to(device)
                labels=labels.to(device)
        
                optimizer.zero_grad()

                outputs = model(inputs)
                
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss  += loss.item()
                running_corrects += torch.sum(preds == labels.data).item()
                
            if phase=="train":
                epoch_loss = running_loss / len(trainloader)
                epoch_acc = running_corrects / len(train_data)
    
                
            else:
                epoch_loss = running_loss / len(validloader)
                epoch_acc = running_corrects / len(valid_data)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [24]:
trained_model= train_model(model, criterion, optimizer,num_epochs=20)

Epoch 1/20
----------
train Loss: 4.4275 Acc: 0.0623
valid Loss: 4.2005 Acc: 0.1137
Epoch 2/20
----------
train Loss: 4.0714 Acc: 0.1645
valid Loss: 3.8436 Acc: 0.1968
Epoch 3/20
----------
train Loss: 3.7432 Acc: 0.2427
valid Loss: 3.4728 Acc: 0.3203
Epoch 4/20
----------
train Loss: 3.4090 Acc: 0.3413
valid Loss: 3.1125 Acc: 0.3863
Epoch 5/20
----------
train Loss: 3.0561 Acc: 0.4107
valid Loss: 2.7275 Acc: 0.4817
Epoch 6/20
----------
train Loss: 2.7453 Acc: 0.4731
valid Loss: 2.4167 Acc: 0.5257
Epoch 7/20
----------
train Loss: 2.4565 Acc: 0.5328
valid Loss: 2.1159 Acc: 0.5990
Epoch 8/20
----------
train Loss: 2.2043 Acc: 0.5972
valid Loss: 1.8489 Acc: 0.6601
Epoch 9/20
----------
train Loss: 2.0059 Acc: 0.6358
valid Loss: 1.6352 Acc: 0.7213
Epoch 10/20
----------
train Loss: 1.7851 Acc: 0.6847
valid Loss: 1.4655 Acc: 0.7433
Epoch 11/20
----------
train Loss: 1.6300 Acc: 0.7063
valid Loss: 1.3131 Acc: 0.7934
Epoch 12/20
----------
train Loss: 1.4880 Acc: 0.7437
valid Loss: 1.1772 A

In [57]:
torch.save(model, "../densenet_model.pth")

In [58]:
model = torch.load("../densenet_model.pth")

In [59]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [78]:
def get_predictions():
    pred_list=[]
    results = {}
    for image, label in testloader:
        optimizer.zero_grad()
        image=image.to(device)
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        pred_list.append(preds.item())
    return pred_list

In [79]:
import os
path='./input/test set/test set'
image_list=os.listdir("./input/test set/test set")
image_list

['image_07615.jpg', 'teste.jpg']

In [82]:
pred_list = get_predictions()
print(pred_list)
for i in range(len(pred_list)):
    print(pred_list[i], all_labels[pred_list[i] - 1], image_list[i])

[15, 39, 15, 45, 61, 33, 92, 92, 40, 39, 45, 15, 92, 45, 75, 28, 21, 56, 91, 45, 55, 92, 14, 15, 75, 17, 35, 39, 55, 14, 92, 56, 75, 89, 78, 28, 92, 17, 92, 15, 56, 45, 45, 90, 45, 90, 28, 32, 15, 92, 78, 75, 92, 100, 15, 75, 78, 28, 92, 92, 56, 100, 92, 45, 28, 28, 56, 56, 56, 75, 0, 89, 28, 45, 75, 75, 63, 50, 89, 75, 56, 89, 15, 89, 96, 45, 92, 100, 32, 15, 56, 92, 45, 92, 89, 17, 45, 78, 63, 56, 92, 75, 28, 75, 45, 75, 75, 15, 35, 75, 89, 28, 15, 28, 92, 94, 32, 92, 75, 75, 75, 99, 92, 17, 75, 15, 17, 17, 75, 45, 15, 25, 45, 17, 89, 92, 39, 17, 45, 63, 45, 15, 92, 78, 75, 28, 15, 92, 89, 92, 100, 56, 57, 14, 92, 75, 56, 92, 17, 56, 89, 75, 92, 45, 17, 89, 55, 75, 56, 35, 56, 75, 78, 39, 19, 17, 15, 39, 75, 77, 17, 28, 15, 19, 45, 35, 45, 75, 75, 28, 0, 63, 17, 75, 17, 92, 45, 75, 56, 55, 28, 75, 92, 92, 89, 17, 15, 15, 39]
15 yellow iris image_07615.jpg
39 siam tulip teste.jpg


IndexError: list index out of range

In [88]:
from PIL import Image
import torchvision.transforms as transforms

def predict(model, image):
    results = {}
    optimizer.zero_grad()
    image = image.to(device)
    outputs = model(image)
    _, preds = torch.max(outputs.data, 1)
    for o in range(len(outputs[0])):
        results[o] = float(outputs[0][o])
        
    #print(f"Camellia score: {float(results['camellia'])}")
    return results

def transform_image(path):
    image = Image.open(path)

    transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                    [0.229, 0.224, 0.225])])

    t = transform(image)

    return t.unsqueeze(0)

In [90]:
i = transform_image('./camelia.jpg')

res = predict(model, i)
res    


{0: 0.07603823393583298,
 1: -0.32939302921295166,
 2: -0.0785537138581276,
 3: 0.037928320467472076,
 4: 0.3027186691761017,
 5: 0.04962935298681259,
 6: 0.25012052059173584,
 7: 0.018258534371852875,
 8: -0.18301913142204285,
 9: 0.013701722025871277,
 10: 0.11280623078346252,
 11: -0.17890332639217377,
 12: -0.13528744876384735,
 13: 0.15092402696609497,
 14: 0.05155510827898979,
 15: 0.47649404406547546,
 16: -0.052039191126823425,
 17: 0.39018309116363525,
 18: 0.23736223578453064,
 19: 0.15677431225776672,
 20: -0.12400052696466446,
 21: 0.07730373740196228,
 22: -0.11392802000045776,
 23: -0.4084746539592743,
 24: -0.30493760108947754,
 25: -0.1102425754070282,
 26: 0.13386359810829163,
 27: 0.06423007696866989,
 28: 0.258838951587677,
 29: 0.33554694056510925,
 30: 0.08153659850358963,
 31: -0.17656397819519043,
 32: -0.024113424122333527,
 33: 0.26168403029441833,
 34: -0.1768830120563507,
 35: 0.3535160422325134,
 36: -0.12726256251335144,
 37: -0.2831449806690216,
 38: -0.35